In [2]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from scipy.spatial import distance
from collections import Counter

In [13]:
# img = cv2.imread('data/R200/R200f_mandela.jpg',cv2.IMREAD_COLOR)
# img2 = cv2.imread('data/R100/R100b_mandela5.jpg',cv2.IMREAD_COLOR)
# #img_blur = cv2.GaussianBlur(img, (5,5), 0)
# img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img_gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
# #img_eq = cv2.equalizeHist(img_gray)
# #img_neg = cv2.bitwise_not(img_eq)
# img_resized = cv2.resize(img_gray, (574,265))
# img_resized2 = cv2.resize(img_gray2, (574,265))



orb = cv2.ORB_create(nfeatures=500)
data_dir = 'data'
classes = ['R10', 'R20', 'R50', 'R100', 'R200']

# print(des1.shape)
# imgKp1 = cv2.drawKeypoints(img_resized, kp1, None)

# bf = cv2.BFMatcher()
# matches = bf.knnMatch(des1, des2, k=2)
# good = []
# for m,n in matches:
#     if m.distance < 0.75*n.distance:
#         good.append([m])
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    img_neg = cv2.bitwise_not(img_eq)
    img_resized = cv2.resize(img_gray, (574, 265))
    edges = cv2.Canny(img_resized, 100, 200)
    return img_resized

def extract_features(img):
    keypoints, descriptors = orb.detectAndCompute(img, None)
    return descriptors

# Process all images and extract features
train_descriptors = []
train_labels = []
for label in classes:
    class_dir = os.path.join(data_dir, label)
    for filename in os.listdir(class_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(class_dir, filename)
            preprocessed_img = preprocess_image(img_path)
            descriptors = extract_features(preprocessed_img)
            if descriptors is not None:
                train_descriptors.append(descriptors)
                train_labels.append(label)

# Preprocess and extract features from the test image
test_img_path = 'test50.jpg'
test_img = preprocess_image(test_img_path)
test_descriptors = extract_features(test_img)
print(test_descriptors.shape)

# Use BFMatcher to match the descriptors to the test image
bf = cv2.BFMatcher()
all_matches = []

for i, train_desc in enumerate(train_descriptors):
    matches = bf.knnMatch(test_descriptors, train_desc, k=2)
    # Apply ratio test
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    all_matches.extend([(train_labels[i], m) for m in good_matches])

k = 23  # Number of nearest neighbors to consider for the final classification
all_matches.sort(key=lambda x: x[1].distance)
top_k_matches = all_matches[:k]

# Count the labels of the top k good matches
matched_labels = [label for label, match in top_k_matches]
most_common_label = Counter(matched_labels).most_common(1)[0][0]

print(f'The predicted class for the input image is: {most_common_label}')
# show preprocessed image
cv2.imshow('image', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()







(484, 32)
The predicted class for the input image is: R50
